In [1]:
#!git clone https://github.com/hachons/trailmet.git

In [2]:
cd /workspace/trailmet/trailmet

/workspace/trailmet/trailmet


## **Chip**

This notebook demonstrates the implementation of the paper - **Chip: CHannel Independence-based Pruning for Compact Neural Networks**

Steps to generate feature maps, calculate channel independence and then compress it are as follows:

Load/create the config file.
Load dataset and create dataloaders.
Load the model along with the weights.
Create Chip object and pass the parameters in the form of a dictionary.
Pass the dataloaders into the compress_model method to obtain the compressed model.
Since this is a demo notebook the number of epochs have been set to 2.

In [3]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models.resnet_chip import resnet_50
import yaml

In [4]:
root = "/workspace/trailmet/trailmet/experiments/pruning/chip"

## Loading the dataset.

In [5]:
from trailmet.datasets.classification import DatasetFactory
data_root = "/content/data_dir"

In [6]:
mkdir /content/data_dir

mkdir: cannot create directory ‘/content/data_dir’: File exists


In [7]:
train_transform = transforms.Compose(
[transforms.ToTensor()])

val_transform = transforms.Compose(
[transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}


cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Getting the configurations.

In [8]:
class CFG:
    def __init__(self):
        self.batch_size = 16
        self.arch = 'resnet_50'
        self.repeat = 5
        self.ci_dir = './CI_resnet_50'
        self.lr_type = 'cos'
        self.learning_rate = 0.01
        self.epochs = 1
        self.num_layers = 53
        self.feature_map_dir = './conv_feature_map'
        self.sparsity = '[0.]+[0.1]*3+[0.35]*16'
        self.label_smooth = 0
        self.device = 'cuda:1'
        self.gpu = '1'
        self.momentum = 0.9
        self.weight_decay = 0.005
        self.lr_decay_step = '30,60'
        self.result_dir = './result'
        self.pretrain_dir = './pretrained_models/resnet_50.pt'
        self.conv_index = torch.tensor(1)
        
args = CFG()
args = args.__dict__

## Creating the dataloaders

In [9]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train'], batch_size=16, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val'], batch_size=16, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test'], batch_size=16, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )

## Loading the model.

In [10]:
model = eval(args['arch'])(sparsity=[0.]*100).to(args['device'])
checkpoint = torch.load(args['pretrain_dir'])
model.load_state_dict(checkpoint)

<All keys matched successfully>

## Creating the method's object followed by compression.

In [11]:
from trailmet.algorithms.prune.chip import Chip

In [12]:
a = Chip(model,{'train': train_loader, 'val': val_loader, 'test': test_loader},**args)

In [ ]:
a.compress_model()

/workspace/trailmet/trailmet/trailmet/algorithms/prune/chip.py:318: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  original_norm = torch.norm(torch.tensor(conv_reshape[i, :, :]), p='nuc').item()
/workspace/trailmet/trailmet/trailmet/algorithms/prune/chip.py:341: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(layer_ci_mean_total)
03/23 08:55:12 AM | args = <trailmet.algorithms.prune.chip.Chip object at 0x7f97a23a7370>
03/23 08:55:12 AM | sparsity:[0.0, 0.1, 0.1, 0.1, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35]
03/23 08:55:12 AM | ==> Building model..


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


03/23 08:55:12 AM | ResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): ModuleList(
    (0): Bottleneck(
      (conv1): Conv2d(64, 41, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(41, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(41, 41, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(41, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv3): Conv2d(41, 230, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(230, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu3): ReLU(inplace=True)
 